# Model Selection

1. Data Preparation
   - Load the data
   - Split into features and target
   - Create train/test split

2. Define Evaluation Metrics
   - Accuracy, Precision, Recall, F1-score for win prediction
   - Mean Absolute Error, Mean Squared Error for score prediction

3. Model Comparison (for win prediction)
   - Logistic Regression
   - Random Forest
   - Gradient Boosting (e.g., XGBoost)
   - Support Vector Machines

4. Model Comparison (for score prediction)
   - Linear Regression
   - Decision Trees
   - Random Forest
   - Gradient Boosting

5. Cross-Validation
   - Implement k-fold cross-validation for each model

6. Hyperparameter Tuning
   - Use GridSearchCV or RandomizedSearchCV for best models

7. Final Model Selection
   - Choose the best model based on cross-validation results
   - Evaluate on the test set

8. Save Best Models
   - Save the best models and their corresponding scalers

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import sqlite3
import pandas as pd
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split

# Add the project root to the Python path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(notebook_dir)
sys.path.append(project_root)

from src.visualization.distribution_plots import visualize_null_values

## 1.   Data Preparation

In [2]:
target_db_path = '../data/04_features/features_teams.db'
conn = sqlite3.connect(target_db_path)

# Read the data into a DataFrame
df_all_years = pd.read_sql_query("SELECT * FROM features_teams", conn)

# Close the connection
conn.close()

In [3]:
def split_data(df, target_column, drop_columns=None, test_size=0.2, val_size=0.2, random_state=42):
    """
    Split the data into train, validation, and test sets based on years.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame
    target_column (str): The name of the target column
    drop_columns (list): List of column names to drop from features. If None, use all columns except target and 'season'
    test_size (float): Proportion of data to use for test set
    val_size (float): Proportion of non-test data to use for validation set
    random_state (int): Random state for reproducibility
    
    Returns:
    tuple: (X_train, X_val, X_test, y_train, y_val, y_test)
    """
    
    # Sort the DataFrame by year to ensure chronological order
    df = df.sort_values('year')
    
    # Define columns to drop
    if drop_columns is None:
        drop_columns = []
    drop_columns = set(drop_columns + [target_column, 'year'])
    
    # Select feature columns (all columns except those in drop_columns)
    feature_columns = [col for col in df.columns if col not in drop_columns]
    
    # Split features (X) and target (y)
    X = df[feature_columns]
    y = df[target_column]
    
    # Get unique years
    years = df['year'].unique()
    
    # Calculate the number of years for test and validation
    n_years = len(years)
    n_test_years = max(1, int(n_years * test_size))
    n_val_years = max(1, int((n_years - n_test_years) * val_size))
    
    # Split years into train, validation, and test
    test_years = years[-n_test_years:]
    val_years = years[-(n_test_years + n_val_years):-n_test_years]
    train_years = years[:-(n_test_years + n_val_years)]
    
    # Create masks for each split
    test_mask = df['year'].isin(test_years)
    val_mask = df['year'].isin(val_years)
    train_mask = df['year'].isin(train_years)
    
    # Split the data
    X_train, y_train = X[train_mask], y[train_mask]
    X_val, y_val = X[val_mask], y[val_mask]
    X_test, y_test = X[test_mask], y[test_mask]
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [4]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(
    df_all_years[df_all_years['year'] < 2024],
    target_column='win',
    drop_columns=[
        'season_type',
        'team_id',
        'opponent_id',
        'team_conference',
        'opponent_conference',
        'start_date',
        'venue_id'
        ]
    )

##  3.  Model Comparison

### 3.1 Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

def create_improved_logistic_regression_model(X_train, X_val, X_test, y_train, y_val, y_test):
    # Create a pipeline with more steps and options
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('logistic_regression', LogisticRegression(random_state=42, max_iter=1000))
    ])
    
    # Define parameter grid for GridSearchCV
    param_grid = {
        'poly__degree': [1],
        'logistic_regression__C': [0.01, 0.1, 1, 10],
        'logistic_regression__penalty': ['l2'],
        'logistic_regression__solver': ['lbfgs']
    }
    
    # Perform grid search
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Get the best model
    best_model = grid_search.best_estimator_
    
    # Make predictions
    train_predictions = best_model.predict(X_train)
    val_predictions = best_model.predict(X_val)
    test_predictions = best_model.predict(X_test)
    
    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, train_predictions)
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    
    # Print results
    print("Best parameters:", grid_search.best_params_)
    print("\nValidation Set Classification Report:")
    print(classification_report(y_val, val_predictions))
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Get feature importances
    feature_importances = abs(best_model.named_steps['logistic_regression'].coef_[0])
    feature_names = X_train.columns
    
    # Print top 10 feature importances
    print("\nTop 10 Feature Importances:")
    for name, importance in sorted(zip(feature_names, feature_importances), key=lambda x: x[1], reverse=True)[:10]:
        print(f"Feature '{name}': {importance:.4f}")
    
    return best_model

# Usage
logistic_model = create_improved_logistic_regression_model(X_train, X_val, X_test, y_train, y_val, y_test)

/Users/colingaffney/repos/personal-projects/college_football_predictor/venv/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['avg_line_spread' 'team_talent' 'opponent_talent']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/colingaffney/repos/personal-projects/college_football_predictor/venv/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['avg_line_spread' 'team_talent' 'opponent_talent']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/colingaffney/repos/personal-projects/college_football_predictor/venv/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['avg_line_spread' 'team_talent' 'opponent_talent']. At least one non-missing value is needed for imputation with strategy

Best parameters: {'logistic_regression__C': 0.01, 'logistic_regression__penalty': 'l2', 'logistic_regression__solver': 'lbfgs', 'poly__degree': 1}

Validation Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.77      0.77      0.77      1485
         1.0       0.77      0.76      0.77      1485

    accuracy                           0.77      2970
   macro avg       0.77      0.77      0.77      2970
weighted avg       0.77      0.77      0.77      2970

Training Accuracy: 0.7613
Validation Accuracy: 0.7680
Test Accuracy: 0.7479

Top 10 Feature Importances:
Feature 'opponent_pregame_elo': 0.7797
Feature 'team_pregame_elo': 0.5897
Feature 'is_home': 0.4234
Feature 'avg_line_spread': 0.3616
Feature 'opponent_recruiting_rank': 0.3026
Feature 'team_recruiting_rank': 0.2725
Feature 'firstDowns_last_3': 0.1324
Feature 'points_allowed_last_3': 0.1280
Feature 'offense_rushing_plays_ppa_last_3': 0.0835
Feature 'offense_rushing_plays_total_pp

### 3.2 Random Forest

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score

def create_lightweight_random_forest(X_train, X_val, X_test, y_train, y_val, y_test):
    # Create a pipeline with reduced complexity
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Handle null values
        ('scaler', StandardScaler()),  # Scale features
        ('rf', RandomForestClassifier(n_estimators=1000, max_depth=6, random_state=42))  # Reduced parameters
    ])
    
    # Fit the model
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    train_predictions = pipeline.predict(X_train)
    val_predictions = pipeline.predict(X_val)
    test_predictions = pipeline.predict(X_test)
    
    # Calculate accuracies
    train_accuracy = accuracy_score(y_train, train_predictions)
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    
    # Print results
    print("\nValidation Set Classification Report:")
    print(classification_report(y_val, val_predictions))
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Feature importance (top 10)
    feature_importance = pipeline.named_steps['rf'].feature_importances_
    feature_names = X_train.columns
    print("\nTop 10 Feature Importances:")
    for name, importance in sorted(zip(feature_names, feature_importance), key=lambda x: x[1], reverse=True)[:10]:
        print(f"Feature '{name}': {importance:.4f}")
    
    return pipeline

# Usage
forest_model = create_lightweight_random_forest(X_train, X_val, X_test, y_train, y_val, y_test)


Validation Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.77      0.74      0.76      1485
         1.0       0.75      0.78      0.77      1485

    accuracy                           0.76      2970
   macro avg       0.76      0.76      0.76      2970
weighted avg       0.76      0.76      0.76      2970

Training Accuracy: 0.7792
Validation Accuracy: 0.7616
Test Accuracy: 0.7430

Top 10 Feature Importances:
Feature 'opponent_pregame_elo': 0.1228
Feature 'opponent_recruiting_rank': 0.0823
Feature 'team_pregame_elo': 0.0804
Feature 'opponent_recruiting_points': 0.0662
Feature 'is_home': 0.0606
Feature 'avg_line_spread': 0.0544
Feature 'team_recruiting_rank': 0.0523
Feature 'team_recruiting_points': 0.0357
Feature 'win_rate_last_10': 0.0343
Feature 'win_rate_last_5': 0.0310


### 3.3 XGBoost

In [7]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

def create_improved_xgboost_model(X_train, X_val, X_test, y_train, y_val, y_test):
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('xgb', XGBClassifier(random_state=42, eval_metric='logloss'))
    ])
    
    param_grid = {
        'xgb__n_estimators': [500, 1000, 1500],
        'xgb__max_depth': [3, 4, 5, 6],
        'xgb__learning_rate': [0.01, 0.05, 0.1],
        'xgb__subsample': [0.8, 0.9, 1.0],
        'xgb__colsample_bytree': [0.8, 0.9, 1.0]
    }
    
    grid_search = RandomizedSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, n_iter=20, random_state=42)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    
    # Make predictions
    y_train_pred = best_model.predict(X_train)
    y_val_pred = best_model.predict(X_val)
    y_test_pred = best_model.predict(X_test)
    
    # Calculate accuracies
    train_accuracy = accuracy_score(y_train, y_train_pred)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    # Print results
    print("Best parameters:", grid_search.best_params_)
    print("\nTraining Accuracy:", train_accuracy)
    print("Validation Accuracy:", val_accuracy)
    print("Test Accuracy:", test_accuracy)
    
    print("\nValidation Set Classification Report:")
    print(classification_report(y_val, y_val_pred))
    
    # Feature importance
    feature_importance = best_model.named_steps['xgb'].feature_importances_
    feature_names = X_train.columns
    feature_importance_dict = dict(zip(feature_names, feature_importance))
    sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
    
    print("\nTop 10 Important Features:")
    for feature, importance in sorted_features[:10]:
        print(f"{feature}: {importance:.4f}")
    
    return best_model

# Usage
xgboost_model = create_improved_xgboost_model(X_train, X_val, X_test, y_train, y_val, y_test)

/Users/colingaffney/repos/personal-projects/college_football_predictor/venv/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['team_talent' 'opponent_talent']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/colingaffney/repos/personal-projects/college_football_predictor/venv/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['team_talent' 'opponent_talent']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/colingaffney/repos/personal-projects/college_football_predictor/venv/lib/python3.12/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['team_talent' 'opponent_talent']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/colingaffney/repos/pe

Best parameters: {'xgb__subsample': 0.8, 'xgb__n_estimators': 500, 'xgb__max_depth': 5, 'xgb__learning_rate': 0.01, 'xgb__colsample_bytree': 0.9}

Training Accuracy: 0.8038783269961978
Validation Accuracy: 0.7643097643097643
Test Accuracy: 0.7451737451737451

Validation Set Classification Report:
              precision    recall  f1-score   support

         0.0       0.77      0.76      0.76      1485
         1.0       0.76      0.77      0.77      1485

    accuracy                           0.76      2970
   macro avg       0.76      0.76      0.76      2970
weighted avg       0.76      0.76      0.76      2970


Top 10 Important Features:
team_pregame_elo: 0.0388
is_home: 0.0353
opponent_pregame_elo: 0.0322
opponent_recruiting_rank: 0.0280
win_rate_last_5: 0.0271
team_recruiting_rank: 0.0266
win_rate_last_1: 0.0189
avg_line_spread: 0.0189
win_rate_last_10: 0.0172
points_allowed_last_3: 0.0112


## Initial Predictions

In [8]:
from src.utils.team_pairs import get_team_pairs
import joblib
from datetime import datetime
import pandas as pd

# Get team pairs
team_pairs = dict(get_team_pairs())

# Filter the data for 2024 week 2 games
df_2024_week2 = df_all_years[(df_all_years['year'] == 2024) & (df_all_years['week'].isin([1, 2, 3, 4]))]

# Prepare the features for prediction
X_predict = df_2024_week2.drop(['win', 'year', 'week', 'season_type', 'team_id', 'opponent_id', 'team_conference', 'opponent_conference', 'start_date'], axis=1)

# Ensure X_predict has the same columns as X_train
missing_cols = set(X_train.columns) - set(X_predict.columns)
for col in missing_cols:
    X_predict[col] = 0

X_predict = X_predict[X_train.columns]

# Predict probabilities for 2024 week 2 games using all three models
logistic_probabilities = logistic_model.predict_proba(X_predict)[:, 1]
forest_probabilities = forest_model.predict_proba(X_predict)[:, 1]
xgboost_probabilities = xgboost_model.predict_proba(X_predict)[:, 1]

# Create the result DataFrame
result_df = pd.DataFrame({
    'year': df_2024_week2['year'],
    'week': df_2024_week2['week'],
    'team_id': df_2024_week2['team_id'],
    'team': df_2024_week2['team_id'].map(team_pairs),
    'opponent_id': df_2024_week2['opponent_id'],
    'opponent': df_2024_week2['opponent_id'].map(team_pairs),
    'is_home': df_2024_week2['is_home'],
    'logistic_win_probability': logistic_probabilities,
    'forest_win_probability': forest_probabilities,
    'xgboost_win_probability': xgboost_probabilities,
    'win_probability': (logistic_probabilities + forest_probabilities + xgboost_probabilities) / 3
})

# Create a pair column
result_df['pair'] = result_df.apply(lambda row: tuple(sorted([row['team_id'], row['opponent_id']])), axis=1)

# Function to process predictions for a pair of teams
def process_team_pair(group):
    home_team = group[group['is_home'] == 1].iloc[0]
    away_team = group[group['is_home'] == 0].iloc[0]
    
    win_prob = (home_team['win_probability'] + (1 - away_team['win_probability'])) / 2
    
    return pd.Series({
        'year': home_team['year'],
        'week': home_team['week'],
        'home_id': home_team['team_id'],
        'home_team': home_team['team'],
        'away_id': away_team['team_id'],
        'away_team': away_team['team'],
        'win_probability': win_prob
    })

# Group by pair and apply the processing function
final_result = result_df.groupby('pair').apply(process_team_pair).reset_index(drop=True)

# Sort by home win probability in descending order
final_result = final_result.sort_values('win_probability', ascending=False)

# Display the result
print(final_result)

# Save the result as a parquet file and models as joblib files
output_dir = '../models/win_probability'
os.makedirs(output_dir, exist_ok=True)

# Generate timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save prediction parquet file
output_file = f'{output_dir}/predictions_{final_result["year"].iloc[0]}_{final_result["week"].iloc[0]}.parquet'
final_result.to_parquet(output_file, index=False)

# Save models
# Generate date (without time)
date = datetime.now().strftime("%Y%m%d")

joblib.dump(logistic_model, f'{output_dir}/logistic_{date}.joblib')
joblib.dump(forest_model, f'{output_dir}/forest_{date}.joblib')
joblib.dump(xgboost_model, f'{output_dir}/xgboost_{date}.joblib')

print(f"Predictions saved to: {output_file}")
print(f"Models saved with timestamp: {date}")

/var/folders/yq/dnnn56yn2l34mgnc3mc_8kyr0000gn/T/ipykernel_14501/3023342378.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_result = result_df.groupby('pair').apply(process_team_pair).reset_index(drop=True)


     year  week  home_id         home_team  away_id         away_team  \
153  2024     4      251             Texas     2433  Louisiana Monroe   
192  2024     3     2633         Tennessee     2309        Kent State   
134  2024     4      213        Penn State     2309        Kent State   
126  2024     1      201          Oklahoma      218            Temple   
53   2024     2       61           Georgia     2635    Tennessee Tech   
..    ...   ...      ...               ...      ...               ...   
100  2024     4     2393  Middle Tennessee      150              Duke   
46   2024     4       58     South Florida     2390             Miami   
194  2024     1     2440            Nevada     2567               SMU   
51   2024     3       96          Kentucky       61           Georgia   
159  2024     3      328        Utah State      254              Utah   

     win_probability  
153         0.960780  
192         0.959750  
134         0.954180  
126         0.952021  
53      